After install dependancies

In [ ]:
pip install selenium

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
import time

In [ ]:
# Start the webdriver and navigate to the website
path = '/chromedriver'
driver = webdriver.Chrome(service=Service(path))
driver.get("https://www.veikkaus.fi/fi/vedonlyonti/pitkaveto?t=1-2-1_Englanti%3B1-2-8_Englanti%3B1-2-7_Englanti%3B1-2-2_Englanti%3B1-2-3_Englanti%3B1-2-4_Englanti%3B1-2-9_Englanti%3B1-2-15_Englanti")


#Accept Coookies consent
cookie_consent_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[@class='iOMuvDyuoMKpVRzmevldA3KUmWI2t3RJ Modal-module_modal__2u5Eo Modal-module_compact__28k8e']/div[2]/button[1]/div[1]/span")))
cookie_consent_button.click()

driver.refresh()


home_team_list = []
away_team_list = []
players_names_list = []
odds_list = []

# Find all buttons to click on to get to the odds page
odds_buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, "//div[@class='pitkaveto-subpage-game-row__bets--extra']/button[1]")))

for i in range(len(odds_buttons)):  
    odds_buttons[i].click()

    # Wait for the page to load
    time.sleep(3)

    # Get the home team and away team names
    home_team = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='header-event-title']/h1/span[1]"))).text
    away_team = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='header-event-title']/h1/span[2]"))).text

    # Print the home team and away team names
    print(f"{home_team} vs {away_team}")

    try:
        # Click on the "Pelaajakohteet" button to view player props
        player_props_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Pelaajakohteet')]")))
        player_props_button.click()

        # Wait for the player props to load
        time.sleep(3)

        # Get the names and odds of the players
        player_names = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='pitkaveto-subpage-sub-rows__content--cards']/div[3]/div[2]/div[1]/div[1]/button/div/span[1]")))
        player_odds = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='pitkaveto-subpage-sub-rows__content--cards']/div[3]/div[2]/div[1]/div[1]/button/div/span[2]")))

        # Print the names and odds of the players
        for j in range(len(player_names)):
            print(f"{player_names[j].text}: {player_odds[j].text}")
    except:
        # If there are no player props available, print a message
        print("No player props available")
    

    # Go back to the previous page
    driver.execute_script("window.history.go(-1)")

    # Wait for the page to load
    time.sleep(3)

    # Find all the odds buttons again
    odds_buttons = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@class='pitkaveto-subpage-game-row__bets--extra']/button[1]")))

# Close the browser
driver.quit()


In [ ]:
mydikt={'player':players_names_list,'Odds':odds_list}
df=pd.DataFrame.from_dict(mydikt)
df.to_excel('scoringodds.xlsx',index=False)
df